MAIN FEATURE

In [ ]:
playlist_link = "https://www.youtube.com/playlist?list=PL0gdCDw52PLWC3Im-OP3CfyJFozvqCSuH"

In [ ]:
from time import sleep
from selenium import webdriver
from tqdm import tqdm


def init(playlist_link):
    browser = webdriver.Chrome()
    browser.get(playlist_link)

    stats = browser.find_element_by_id("stats")
    num_of_videos = stats.text.split()[0]
    play_all_button = browser.find_element_by_xpath(
        '//*[@class="style-scope ytd-playlist-sidebar-primary-info-renderer"]')
    play_all_button.click()
    sleep(2)
    mute_button = browser.find_element_by_xpath(
        '//*[@class="ytp-mute-button ytp-button"]')
    mute_button.click()
    return browser, num_of_videos

def scrape_titles():
    titles = []
    for _ in tqdm(range(int(num_of_videos))):
        title = browser.find_element_by_xpath(
            '//*[@class="style-scope ytd-video-primary-info-renderer"]') \
            .text.splitlines()[0]        
        titles.append(title)
        next_button = browser.find_element_by_xpath(
            '//*[@class="ytp-next-button ytp-button"]')
        browser.execute_script("arguments[0].click();", next_button)
        sleep(2) 
        
    browser.close()
    return titles


browser, num_of_videos = init(playlist_link)
titles = scrape_titles()

titles


In [ ]:
titles_as_URL = [title.replace(" ", "%20") for title in titles]

titles_as_URL

In [ ]:
spotify_username = "113963150"
spotify_api_token = "BQAA-jyLc4lhy3WJusULLAi3QW5rQqGbe85pTHfUHYENzJQ4JRgVAGSm41iI2sNdqoudXER8ltlu8jjxx_TStfz4lfKnPCNRWv_8Dl1VRB0QXDlRICZUaVTbc6lQRYVYw1KrMT3DO-7v-i2q4b96m6bTYQJ8SfvPRIm7JZuzDPaUMCmSFBguIgP1lzOsMUS-wqmdsbT87gM"
playlist_name = "YouTube Playlist Exam Edition"
playlist_description = "Created from Python script"

In [ ]:
import requests
import json

def create_playlist(user_id, token, playlist_name, playlist_description):
    url_create_playlist = "https://api.spotify.com/v1/users/" + \
                          f"{user_id}/playlists"
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}"
    }
    body = {
        "name": playlist_name,
        "description": playlist_description
    }
    r = requests.post(url_create_playlist, 
                      headers=headers,
                      data=json.dumps(body))
    results = r.json()
    return results["id"]

playlist_id = create_playlist(spotify_username, 
                                spotify_api_token,
                                playlist_name, 
                                playlist_description)
playlist_id

In [ ]:
def find_spotify_songs(titles_as_URL, token):
    uris = []
    unknown_songs = []
    for url_title in titles_as_URL:
        url = f"https://api.spotify.com/v1/search?q={url_title}&type=track"
        headers = {
            "Accept": "application/json",
            "Content-Type": "application/json",
            "Authorization": f"Bearer {token}"
        }
        r = requests.get(url, headers=headers)
        results = r.json()
        try:
            uris.append(results["tracks"]["items"][0]["uri"])
        except KeyError:
            if results["error"]:
                print(results["error"]["message"])
        except IndexError:
            unknown_songs.append(url_title.replace("%20", " "))
    return (uris, unknown_songs)


uris, unknown_songs = find_spotify_songs(titles_as_URL, spotify_api_token)
uris

In [ ]:
def add_to_playlist(user_id, uris, token, playlist_id):
    url = f"https://api.spotify.com/v1/users/{user_id}/playlists/" + \
          f"{playlist_id}/tracks"
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}"
    }
    body = {
        "uris": uris
    }
    requests.post(url, headers=headers,
                       data=json.dumps(body))
                       
        
add_to_playlist(spotify_username, uris, spotify_api_token, playlist_id)

In [ ]:
if len(unknown_songs) > 0:
    print("The following songs couldn't be added to the playlist:")
    for song in unknown_songs:
        print(song)